In [ ]:
# run(`conda create --channel conda-forge --channel bioconda --channel defaults --strict-channel-priority --name blast blast`)

In [1]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [2]:
data_dir = joinpath(dirname(pwd()), "data")

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data"

In [3]:
memory=64
threads=max(1, Int(memory//8))

8

In [4]:
blast_task = "megablast"

"megablast"

In [5]:
db = "nt"

"nt"

In [6]:
blastdbs_dir = mkpath(joinpath(homedir(), "workspace", "blastdbs"))

"/home/cjprybol/workspace/blastdbs"

In [22]:
blast_db = joinpath(blastdbs_dir, db)

"/home/cjprybol/workspace/blastdbs/nt"

In [ ]:
Mycelia.download_blast_db(db="nt", outdir=blastdbs_dir, source="ncbi", conda_env="blast")

In [8]:
# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213
┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213


Row,Experiment Accession,Experiment Title,Organism Name,Instrument,Submitter,Study Accession,Study Title,Sample Accession,Sample Title,"Total Size, Mb",Total RUNs,Total Spots,Total Bases,Library Name,Library Strategy,Library Source,Library Selection,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,LibraryName,LibraryLayout,InsertSize,InsertDev,Platform,BioProject,Study_Pubmed_id,ProjectID,BioSample,SampleType,TaxID,g1k_pop_code,source,g1k_analysis_group,Subject_ID,Sex,Disease,Tumor,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash,+access,+submission_date,+publication_date,+last_update,altitude,env_feature,collection_date,sample number,sample_type,env_biome,lat_lon,geo_loc_name,isolation source,host,env_material,location,geo,geo2,duration,date.month,date.start,date.end,season,particle,temperature,humid,weekend,aownership,latitude,longitude
,String,String,String,String,String,String,String,String,String,Float64,Int64,Int64,Int64,String,String,String,String,String,String,String,Int64,Int64,Int64,Int64,Int64,String,String,String,String,Int64,Int64,String,String,String,Int64,String,String,Int64,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,Int64,String,String,String,String,String,String,String,String,String,String,String,Int64,String,String,String,String,String,String,String,String,String,String
1,SRX3492741,DNA-seq of air samples,air metagenome,Illumina HiSeq 4000,Stanford University,SRP127288,Airborne exposures samples Raw sequence reads,SRS2776343,,7161.87,1,58285750,17136430544,chaoexpo67_I5,WGS,METAGENOMIC,MDA,SRR6399706,2018-09-20 00:09:39,2017-12-20 22:41:12,58285750,17136430544,58285750,294,7161,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos5/sra-pub-zq-14/SRR006/399/SRR6399706.sralite.1,chaoexpo67_I5,PAIRED,0,0,ILLUMINA,PRJNA421162,,421162,SAMN08215973,simple,655179,,,,,,,no,,,,,STANFORD UNIVERSITY,SRA640713,,public,A7D306A38C3D399BA83DE62FB6ABB8E1,F36CEBD819C2CC9B604A802F16E1314B,public,2017-12-20T15:50:13.253,2018-09-20T00:00:00.000,2021-02-28T05:15:11.191,51,air filter,2017-02-02,283,DNA,air filter,37.40988969 N 122.1593497 W,"USA:california,Palo Alto",missing,missing,air,Campus,uswest,Westcoast,2.403587963,1,2017-01-28,2017-02-02,winter,6.433836664,25.00845376,34.67120431,weekday,P1,37.40988969,-122.1593497
2,SRX3492740,DNA-seq of air samples,air metagenome,Illumina HiSeq 4000,Stanford University,SRP127288,Airborne exposures samples Raw sequence reads,SRS2776345,,6749.48,1,51048135,14967992451,chaoexpo67_I6,WGS,METAGENOMIC,MDA,SRR6399707,2018-09-20 00:09:39,2017-12-20 23:30:31,51048135,14967992451,51048135,293,6749,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos5/sra-pub-zq-14/SRR006/399/SRR6399707.sralite.1,chaoexpo67_I6,PAIRED,0,0,ILLUMINA,PRJNA421162,,421162,SAMN08215972,simple,655179,,,,,,,no,,,,,STANFORD UNIVERSITY,SRA640713,,public,F7C637A0ECA050584FDD0B5EAB344209,4E7EC27C92CEE1C8DE1265D86B4E6F9C,public,2017-12-20T15:50:13.240,2018-09-20T00:00:00.000,2021-02-28T05:15:11.060,5,air filter,2017-01-27,282,DNA,air filter,29.272251 N 94.818915 W,"USA:texas,Galveston",missing,missing,air,Houston-TX,uswest,Southwest,2.953472222,1,2017-01-24,2017-01-27,winter,4.387703582,24.65875088,39.45660318,weekday,P1,29.272251,-94.818915
3,SRX3492739,DNA-seq of air samples,air metagenome,Illumina HiSeq 4000,Stanford University,SRP127288,Airborne exposures samples Raw sequence reads,SRS2776338,,7797.92,1,87571718,25550913488,chaoexpo67_I15,WGS,METAGENOMIC,MDA,SRR6399708,2018-09-20 00:09:39,2017-12-21 23:57:29,87571718,25550913488,87571718,291,7797,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos5/sra-pub-zq-14/SRR006/399/SRR6399708.sralite.1,chaoexpo67_I15,PAIRED,0,0,ILLUMINA,PRJNA421162,,421162,SAMN08215971,simple,655179,,,,,,,no,,,,,S

In [9]:
run_ids = sort(joint_metadata[!, "Run"])

213-element Vector{String}:
 "SRR6399459"
 "SRR6399460"
 "SRR6399461"
 "SRR6399462"
 "SRR6399464"
 "SRR6399485"
 "SRR6399486"
 "SRR6399487"
 "SRR6399488"
 "SRR6399490"
 "SRR6399491"
 "SRR6399492"
 "SRR6399494"
 ⋮
 "SRR6399977"
 "SRR6399986"
 "SRR6399988"
 "SRR6399991"
 "SRR6400014"
 "SRR6400015"
 "SRR6400017"
 "SRR6400018"
 "SRR6400019"
 "SRR6400022"
 "SRR6400023"
 "SRR6400024"

In [10]:
sample_paths = joinpath.(data_dir, "SRA", run_ids)

213-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399459"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399460"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399461"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399462"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399464"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399485"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399486"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399487"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399488"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399490"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/SRR6399491"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 39 bytes ⋯ "e-discovery/data/SRA/

In [ ]:
# sample_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
# sample_paths = filter(x -> isfile(joinpath.(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

In [23]:
for sample_path in sample_paths[1:1]
    sample = basename(sample_path)
    blastn_directory = joinpath(sample_path, "blastn")
    assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
    target_outfile = joinpath(blastn_directory, basename(assembled_fasta) * ".blastn.$(db).$(blast_task).txt")
    # if !isfile(target_outfile)
    # @show sample
    cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db $(blast_db) --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
    submission = 
    `sbatch 
    --mail-user=cameron.prybol@gmail.com
    --mail-type=ALL
    --error=%x-%j.err
    --output=%x-%j.out
    --partition=batch
    --account=mpsnyder
    --nodes=1
    --ntasks=1
    --time=1-00:00:00    
    --cpus-per-task=$(threads)
    --mem=$(memory)G
    --job-name=blastn
    --wrap $cmd
    `
    run(submission)
    # end
end

Submitted batch job 42295281


In [ ]:
# findlast(sample_path -> !isempty(readdir(joinpath(sample_path, "blastn"))), sample_paths)

In [ ]:
# sample_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
# sample_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

# for sample_path in sample_paths
#     sample = basename(sample_path)
#     blastn_directory = joinpath(sample_path, "blastn")
#     @show readdir(blastn_directory)
#     target_outfile = joinpath(sample_path, "blastn", "final.contigs.fastg.gfa.fna.blastn.nt.$(blast_task).txt")
#     # || filesize(target_outfile) == 0
#     if !isfile(target_outfile) 
#         assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
#         cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db nt --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
#         submission = 
#         `sbatch 
#         --mail-user=cameron.prybol@gmail.com
#         --mail-type=ALL
#         --error=%x-%j.err
#         --output=%x-%j.out
#         --partition=batch
#         --account=mpsnyder
#         --nodes=1
#         --ntasks=1
#         --time=1-00:00:00    
#         --cpus-per-task=$(threads)
#         --mem=$(memory)G
#         --job-name=blastn
#         --wrap $cmd
#         `
#         run(submission)
#         # end
#     end
# end